In [21]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import io
from PIL import Image
import glob
import os
import matplotlib
matplotlib.style.use('dark_background')

# Define a function to load and preprocess data from npz files
def load_and_preprocess_data(folder_path):
    data = {}
    files = sorted(glob.glob(os.path.join(folder_path, "*_posture*.npz")))
    min_frame = np.inf
    max_frame = -np.inf
    screen = [np.inf, np.inf, -np.inf, -np.inf]
    #print(files)

    for f in files:
        #print("loading", f)
        data[f] = {}
        with np.load(f) as npz:
            midline = {}
            offset = npz["offset"]
            frames = npz["frames"]

            if min_frame > frames.min():
                min_frame = frames.min()
            if max_frame < frames.max():
                max_frame = frames.max()

            if offset.T[0].min() < screen[0]:
                screen[0] = offset.T[0].min()
            if offset.T[1].min() < screen[1]:
                screen[1] = offset.T[1].min()

            if offset.T[0].max() > screen[2]:
                screen[2] = offset.T[0].max()
            if offset.T[1].max() > screen[3]:
                screen[3] = offset.T[1].max()

            midline = {}
            if len(npz["midline_points"].shape) == 2:
                i = 0
                indices = []
                for l in npz["midline_lengths"][:-1]:
                    i += l
                    indices.append(int(i))
                points = np.split(npz["midline_points"], indices, axis=0)
                for frame, point, off in zip(frames, points, offset):
                    midline[frame] = point + off
            else:
                for mpt, off, frame in zip(npz["midline_points"], offset, frames):
                    midline[frame] = mpt + off

            i = 0
            indices = []
            for l in npz["outline_lengths"][:-1]:
                i += l
                indices.append(int(i))
            points = np.split(npz["outline_points"], indices, axis=0)
            outline = {}
            for frame, point, off in zip(frames, points, offset):
                outline[frame] = point + off

            # Handle holes
            hole_counts = npz["hole_counts"].astype(int)
            hole_points = npz["hole_points"]
            holes = {}
            count_index = 0
            point_index = 0

            for frame in frames:
                holes[frame] = []
                num_holes = hole_counts[count_index]
                count_index += 1
                
                for _ in range(num_holes):
                    num_points = hole_counts[count_index]
                    count_index += 1
                    
                    if point_index + num_points > len(hole_points):
                        raise Exception(f"Error: index {point_index + num_points} is out of bounds for hole_points with size {len(hole_points)}")
                        break
                    
                    #print(f"{frame}: {num_points} points at index {point_index}")
                    hole = hole_points[point_index:point_index + num_points]
                    holes[frame].append(hole)
                    point_index += num_points

            data[f]["holes"] = holes
            data[f]["midline"] = midline
            data[f]["outline"] = outline

    screen[0] -= 10
    screen[1] -= 10
    screen[2] *= 1.1
    screen[3] *= 1.1
    input_shape = (screen[2] - screen[0], screen[3] - screen[1])
    output_width = 1280
    output_shape = (output_width, int(output_width * input_shape[1] / input_shape[0]))  # Adjust output resolution to maintain aspect ratio
    fps = 40.0

    return data, screen, input_shape, output_shape, fps, min_frame, max_frame

import matplotlib.patches as patches
import matplotlib.cm as cm

def calculate_polygon_area(points):
    """Calculate the area of a polygon given its vertices using the shoelace formula."""
    x = points[:, 0]
    y = points[:, 1]
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

def create_video(data, screen, output_shape, fps, min_frame, max_frame, frame_range=None):
    fourcc = cv.VideoWriter_fourcc(*'MJPG')
    filename = "output.avi"
    out = cv.VideoWriter(filename, fourcc, fps, output_shape, True)

    print("Writing video '" + filename + "' with frames", min_frame, "-", max_frame)

    dpi = 250 / 4
    cv.destroyAllWindows()

    frames = np.arange(min_frame, max_frame + 1)
    if frame_range:
        frames = frames[np.logical_and(frames >= frame_range[0], frames <= frame_range[1])]

    plt.style.use('dark_background')
    colormap = cm.get_cmap('cool')

    for chosen_frame in frames:
        fig, ax = plt.subplots(figsize=(output_shape[0] / dpi, output_shape[1] / dpi), dpi=dpi)
        fig.set_tight_layout(True)

        for key in data:
            if chosen_frame not in data[key]["outline"]:
                continue

            outline = np.array(data[key]["outline"][chosen_frame])
            ax.scatter(outline.T[0], outline.T[1], label="outline", s=1.5, color=colormap(0.6))

            midline = data[key]["midline"]
            m = midline[chosen_frame]
            ax.scatter(m.T[0], m.T[1], label="midline", s=1.5, color=colormap(0.9))

            # Draw holes
            hole_area = 0
            for hole in data[key]["holes"][chosen_frame]:
                ax.scatter(hole.T[0], hole.T[1], label="hole", s=1, color='grey')
                hole_area += calculate_polygon_area(hole)

            # Calculate outline area and net area
            outline_area = calculate_polygon_area(outline)
            net_area = outline_area - hole_area

            # Display the area description near the outline
            centroid_x, centroid_y = outline.mean(axis=0)
            ax.text(centroid_x, centroid_y, f"Area: {outline_area:.0f}px²\nNet Area: {net_area:.0f}px²",
                    fontsize=8, color='lightgrey', 
                    bbox=dict(facecolor='black', alpha=0.7, edgecolor='white', boxstyle='round,pad=0.5'))

        ax.set_xlim(screen[0], screen[2])
        ax.set_ylim(screen[3], screen[1])  # Invert y-axis

        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        im = Image.open(buf)
        im = np.array(im).astype(np.uint8)
        buf.close()
        plt.close(fig)

        if int(chosen_frame) % int((frames.max() - frames.min()) * 0.1) == 0:
            print(chosen_frame, "/", frames.max())

        if (im.shape[1], im.shape[0]) != output_shape:
            print("different shape", output_shape, im.shape)

        im = im[:, :, 0:3]
        out.write(im)
        cv.imshow("movie", im)
        cv.waitKey(1)

    out.release()
    print("Video creation complete.")

# Example usage
folder_path = "/path/to/data/"
frame_range = [1000, 2500]  # Example frame range, adjust as needed

data, screen, input_shape, output_shape, fps, min_frame, max_frame = load_and_preprocess_data(folder_path)
create_video(data, screen, output_shape, fps, min_frame, max_frame, frame_range)

Writing video 'output.avi' with frames 0 - 7375


/var/folders/qw/tls_h_cx4k11qqvkl3z991v00000gn/T/ipykernel_12674/1149124948.py:132: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap('cool')


1050 / 2500
1200 / 2500
1350 / 2500
1500 / 2500
1650 / 2500
1800 / 2500
1950 / 2500
2100 / 2500
2250 / 2500
2400 / 2500
Video creation complete.
